In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from nltk import download
from nltk.corpus import stopwords
from wordcloud import WordCloud

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
path = '/content/gdrive/MyDrive/DSA-ICON/'

import os
os.listdir(path)

In [ ]:
df = pd.read_csv(path + 'toyota.csv')
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.duplicated().any()

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)
df

In [ ]:
df.info()

In [ ]:
price = []
for i in df['price']:
  if 0 <= i <= 20000:
    price.append('cheap')
  elif 20001 <= i <= 40000:
    price.append('middle')
  else:
    price.append('expensive')

df['price_category'] = price

In [ ]:
df.head()

In [ ]:
download('stopwords')

stop = stopwords.words('indonesian')

In [ ]:
stop.extend([' '])

In [ ]:
df['model'] = df['model'].str.lower()
df['year'] = [str(year) for year in df['year']]
df['transmission'] = df['transmission'].str.lower()
df['fuelType'] = df['fuelType'].str.lower()

df['model'] = df['model'].apply(
    lambda x: ' '.join([word for word in x.split()
    if word not in (stop)]))

In [ ]:
df['transmission'].unique()

In [ ]:
df['fuelType'].unique()

In [ ]:
sns.set(rc={'figure.figsize':(11.7, 8.27)})

sns.displot(df['mpg'], bins=10)

In [ ]:
sns.set(rc={'figure.figsize':(11.7, 8.27)})

sns.displot(df['price'], bins=10)

In [ ]:
sns.set(rc={'figure.figsize':(11.7, 8.27)})

sns.displot(df['tax'], bins=10)

In [ ]:
sns.set(rc={'figure.figsize':(11.7, 8.27)})

sns.displot(df['engineSize'], bins=10)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

In [ ]:
le = preprocessing.LabelEncoder()

df['model'] = le.fit_transform(df['model'])
df['year'] = le.fit_transform(df['year'])
df['transmission'] = le.fit_transform(df['transmission'])
df['fuelType'] = le.fit_transform(df['fuelType'])

In [ ]:
df 

In [ ]:
X = df.drop(columns=['price_category', 'price'])
y = df['price_category']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 22)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
model = knn.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
from joblib import dump

dump(knn, 'Model_price.pkl')

In [ ]:
import streamlit as st

from pyngrok import ngrok
from joblib import load

trained_model = load("/content/Model_price.pkl")

form = st.form(key='my-form')

Model = st.sidebar.selectbox("Masukkan model: ", ('gt86', 'corolla', 'rav4', 'yaris', 'auris', 'aygo', 'c-hr',
       'prius', 'avensis', 'verso', 'hilux', 'proace verso',
       'land cruiser', 'supra', 'camry', 'verso-s', 'iq', 'urban cruiser'))
year = st.slider("Masukkan tahun: ", 1998, 2020)
trans = st.sidebar.selectbox("Masukkan transmission: ", ('manual', 'automatic', 'semi-auto', 'other'))
mile = form.text_input("Masukkan mileage: ")
fuel = st.sidebar.selectbox("Masukkan fuelType: ", ('petrol', 'other', 'hybrid', 'diesel'))
tax = form.text_input("Masukkan tax: ")
mpg = st.slider("Masukkan mpg: ", 2.8, 235.0)
engSize = st.slider("Masukkan engineSize: ", 0.0, 4.5)

submit = form.form_submit_button("Bagaimana dengan harganya")

if submit:
  hasil_akhir = le.inverse_transform(trained_model.predict([[
    str(Model),
    str(year),
    str(trans),
    int(mile),
    str(fuel),
    int(tax),
    float(mpg), 
    float(engSize)]]))
  st.write("Hasil harga:")
  st.write(str(hasil_akhir))

ngrok_tunnel = ngrok.connect(8501)
print('Public URL:', ngrok_tunnel.public_url)

In [ ]:
!streamlit run harga_mobil.py